# Lab 5: Human in the Loop


## Setup and Initialization

We start by setting up our environment and importing necessary modules. A crucial component is the checkpointer:

```python
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
```

The checkpointer functions similarly to a distributed version control system like Git, but for AI states. It allows us to create "commits" of our AI's state at different points in its decision-making process, and "branch" or "revert" as needed.

Be aware that excessive use of check-pointing can lead to significant memory usage, especially with large state objects. Implement a strategy to manage or expire old checkpoints in long-running applications. Think of usage of Time-To-Live for those checkpoints to be implemented in your database, if you no longer have a need for those checkpoints.
We will be using SQLite as our database of choice for our checkpoints, but you might want to switch to either Redis or Postgres for your production applications.


In [ ]:
from dotenv import load_dotenv
import os
import sys
import json, re
import pprint
import boto3
from botocore.client import Config
import warnings

warnings.filterwarnings("ignore")
import logging

# import local modules
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Set basic configs
logger = utils.set_logger()
pp = utils.set_pretty_printer()

# Load environment variables from .env file or Secret Manager
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")
tavily_ai_api_key = utils.get_tavily_api("TAVILY_API_KEY", aws_region)

# Set bedrock configs
bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Create a bedrock runtime client
bedrock_rt = boto3.client(
    "bedrock-runtime", region_name=aws_region, config=bedrock_config
)

# Create a bedrock client to check available models
bedrock = boto3.client("bedrock", region_name=aws_region, config=bedrock_config)


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_aws import ChatBedrockConverse
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

Next, we set up our agent state with a custom message handling function:


In [ ]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""


def reduce_messages(
    left: list[AnyMessage], right: list[AnyMessage]
) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

This function acts like a smart message broker in a microservices architecture. It ensures that our conversation history remains consistent and up-to-date, crucial for maintaining context in long-running AI interactions.

## Tool and Agent Setup

We integrate the Tavily search tool:


In [ ]:
tool = TavilySearchResults(max_results=2)

## Human Intervention in the Agent Class

The `Agent` class implements several key features that enable human intervention in the AI's decision-making process:

1. **Interrupt Before Action**:

   ```python
   self.graph = graph.compile(
       checkpointer=checkpointer, interrupt_before=["action"]
   )
   ```

   This line sets up an interrupt before the "action" node. It allows human oversight before any action is taken, providing an opportunity for review or modification.
   You can add interrupts before any node you wish for as we will see in the next lab.

2. **State Examination**:
   The `exists_action` method prints the current state:

   ```python
   def exists_action(self, state: AgentState):
       print(state)
       # ...
   ```

   This allows humans to inspect the AI's current state, including its reasoning and intended actions.

3. **Action Visibility**:
   In the `take_action` method, each tool call is printed:

   ```python
   print(f"Calling: {t}")
   ```

   This provides visibility into what actions the AI is about to take, allowing for potential human intervention.

4. **Checkpointing**:
   The `checkpointer` parameter in the constructor allows for saving and loading states. This enables "time travel" functionality, where humans can revisit and potentially modify previous decision points.

5. **Modifiable State**:
   The `AgentState` is a mutable structure. While not shown in this class directly, it allows for state modification, enabling humans to alter the AI's context or decisions.

These features collectively create a framework for human-in-the-loop AI, where human operators can monitor, intervene, and guide the AI's decision-making process at critical junctures.


In [ ]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_bedrock)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_bedrock(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!")
        return {"messages": results}

The `interrupt_before=["action"]` parameter implements a critical control point in our AI pipeline.

It's analogous to implementing approval gates in a CI/CD pipeline, ensuring that no critical action is taken without necessary checks.

To deepen your understanding of AI safety and control mechanisms, explore the paper ["Concrete Problems in AI Safety"](https://arxiv.org/abs/1606.06565) from researchers at Google, Stanford, and Berkeley from 2016.

## Initializing and Running the Agent

We initialize our agent with Claude on Amazon Bedrock:


In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatBedrockConverse(
    client=bedrock_rt,
    model="anthropic.claude-3-haiku-20240307-v1:0",
    temperature=0,
    max_tokens=None,
)
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [ ]:
messages = [HumanMessage(content="Whats the weather in Berlin?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

In [ ]:
thread

## Examining and Modifying State

The ability to examine and modify the agent's state is a powerful feature:


In [ ]:
abot.graph.get_state(thread)

This capability is similar to having a live debugger in a production environment, allowing you to inspect and modify the state of a running process.


In [ ]:
abot.graph.get_state(thread).next

As you can see, the next node to be executed is the `('action',)` node.

Remember, we compiled our graph with the interrupt set before the `action` node.

```python
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )
```


### ...continue after interrupt

Now that stopped before the action (calling tavily-ai), we can continue. Lets see how that works.


In [ ]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

In [ ]:
abot.graph.get_state(thread)

In [ ]:
abot.graph.get_state(thread).next

### Adding your human input

Next, we will see how to add your approval for executing the `action` to search the web.

Depending if you are running in JupyterLab or in e.g. VS Code, the input box should pop up below the code cell, or on the top.

![input box example, asking for user input](../assets/lab5_1.png)


If you are fine with what will be searched, then please add a `y` for yes. Anything else, will abort the operation.


In [ ]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread), "\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

As you can see, the next node to be executed is the `('action',)` node.

Remember, we compiled our graph with the interrupt set before the `action` node.

```python
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )
```

---


Now one thing that can be very interesting, is to only stop and ask for human input, whenever you are calling a certain set of tools, and not all of them.

Think about how that could be achieved, before continue reading.


### Stop at only specific tool calls

1. Parsing the tool call and stopping execution only if the tool call parameter `name` (tool name) is the same as as the stopping parameter.
2. Adding all tools where you need to stop to an extra node.

In general, option 2 is a cleaner and easier to debug. If you would like to see how this is implemented in a bigger example, please head over to the [customer support agent](https://langchain-ai.github.io/langgraph/tutorials/customer-support/customer-support/#part-3-conditional-interrupt) from the LangGraph examples to see how sensitive tools are handled.

Here is a sneak peak of the graph:

![customer support agent graph with sensitive and safe tools](../assets/lab5_2.png)


## Modify State

Run until the interrupt and then modify the state.


In [ ]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

In [ ]:
abot.graph.get_state(thread)

In [ ]:
current_values = abot.graph.get_state(thread)

In [ ]:
current_values.values["messages"][-1]

In [ ]:
current_values.values["messages"][-1].tool_calls

In [ ]:
_id = current_values.values["messages"][-1].tool_calls[0]["id"]
current_values.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in Munich"},
        "id": _id,
    }
]

In [ ]:
abot.graph.update_state(thread, current_values.values)

In [ ]:
abot.graph.get_state(thread)

In [ ]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

## Time Travel


In [ ]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print("--")
    states.append(state)

To fetch the same state as was filmed, the offset below is changed to `-3` from `-1`. This accounts for the initial state `__start__` and the first state that are now stored to state memory with the latest version of software.


In [ ]:
to_replay = states[-3]

In [ ]:
to_replay

In [ ]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

## Go back in time and edit


In [ ]:
to_replay

In [ ]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]
to_replay.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in LA, accuweather"},
        "id": _id,
    }
]

In [ ]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

In [ ]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

## Add message to a state at a given time


In [ ]:
to_replay

In [ ]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]

In [ ]:
# Lets update the humidity to something that is impossible

state_update = {
    "messages": [
        ToolMessage(
            tool_call_id=_id,
            name="tavily_search_results_json",
            content="23 degree celcius, 110 percent humidity",
        )
    ]
}

In [ ]:
branch_and_add = abot.graph.update_state(
    to_replay.config, state_update, as_node="action"
)

In [ ]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

# Extra Practice


## Build a small graph

This is a small simple graph you can tinker with if you want more insight into controlling state memory.


In [ ]:
from dotenv import load_dotenv

_ = load_dotenv()

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator

Define a simple 2 node graph with the following state: -`lnode`: last node -`scratch`: a scratchpad location -`count` : a counter that is incremented each step


In [ ]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [ ]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {
        "lnode": "node_1",
        "count": 1,
    }


def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {
        "lnode": "node_2",
        "count": 1,
    }

The graph goes N1->N2->N1... but breaks after count reaches 3.


In [ ]:
def should_continue(state):
    return state["count"] < 3

In [ ]:
builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)

builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", should_continue, {True: "Node1", False: END})
builder.set_entry_point("Node1")

In [ ]:
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

### Run it!

Now, set the thread and run!


In [ ]:
thread = {"configurable": {"thread_id": str(1)}}
graph.invoke({"count": 0, "scratch": "hi"}, thread)

### Look at current state


Get the current state. Note the `values` which are the AgentState. Note the `config` and the `thread_ts`. You will be using those to refer to snapshots below.


In [ ]:
graph.get_state(thread)

View all the statesnapshots in memory. You can use the displayed `count` agentstate variable to help track what you see. Notice the most recent snapshots are returned by the iterator first. Also note that there is a handy `step` variable in the metadata that counts the number of steps in the graph execution. This is a bit detailed - but you can also notice that the _parent_config_ is the _config_ of the previous node. At initial startup, additional states are inserted into memory to create a parent. This is something to check when you branch or _time travel_ below.


### Look at state history


In [ ]:
for state in graph.get_state_history(thread):
    print(state, "\n")

Store just the `config` into an list. Note the sequence of counts on the right. `get_state_history` returns the most recent snapshots first.


In [ ]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values["count"])

Grab an early state.


In [ ]:
states[-3]

This is the state after Node1 completed for the first time. Note `next` is `Node2`and `count` is 1.


In [ ]:
graph.get_state(states[-3])

### Go Back in Time

Use that state in `invoke` to go back in time. Notice it uses states[-3] as _current_state_ and continues to node2,


In [ ]:
graph.invoke(None, states[-3])

Notice the new states are now in state history. Notice the counts on the far right.


In [ ]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values["count"])

You can see the details below. Lots of text, but try to find the node that start the new branch. Notice the parent _config_ is not the previous entry in the stack, but is the entry from state[-3].


In [ ]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state, "\n")

### Modify State

Let's start by starting a fresh thread and running to clean out history.


In [ ]:
thread2 = {"configurable": {"thread_id": str(2)}}
graph.invoke({"count": 0, "scratch": "hi"}, thread2)

In [ ]:
from IPython.display import Image

Image(graph.get_graph().draw_png())

In [ ]:
states2 = []
for state in graph.get_state_history(thread2):
    states2.append(state.config)
    print(state.config, state.values["count"])

Start by grabbing a state.


In [ ]:
save_state = graph.get_state(states2[-3])
save_state

Now modify the values. One subtle item to note: Recall when agent state was defined, `count` used `operator.add` to indicate that values are _added_ to the current value. Here, `-3` will be added to the current count value rather than replace it.


In [ ]:
save_state.values["count"] = -3
save_state.values["scratch"] = "hello"
save_state

Now update the state. This creates a new entry at the _top_, or _latest_ entry in memory. This will become the current state.


In [ ]:
graph.update_state(thread2, save_state.values)

Current state is at the top. You can match the `thread_ts`.
Notice the `parent_config`, `thread_ts` of the new node - it is the previous node.


In [ ]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  # print latest 3
        break
    print(state, "\n")

### Try again with `as_node`

When writing using `update_state()`, you want to define to the graph logic which node should be assumed as the writer. What this does is allow th graph logic to find the node on the graph. After writing the values, the `next()` value is computed by travesing the graph using the new state. In this case, the state we have was written by `Node1`. The graph can then compute the next state as being `Node2`. Note that in some graphs, this may involve going through conditional edges! Let's try this out.


In [ ]:
graph.update_state(thread2, save_state.values, as_node="Node1")

In [ ]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  # print latest 3
        break
    print(state, "\n")

`invoke` will run from the current state if not given a particular `thread_ts`. This is now the entry that was just added.


In [ ]:
graph.invoke(None, thread2)

Print out the state history, notice the `scratch` value change on the latest entries.


In [ ]:
for state in graph.get_state_history(thread2):
    print(state, "\n")

Continue to experiment!
